## 读取文本

In [1]:
with open("./bbc/business/005.txt","r") as f:
    textString=f.read()

## 文本规范化

In [2]:
import nltk
import re
import string
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also']
# 分词
def tokenize_text(text):
    tokens=nltk.word_tokenize(text)#分词
    tokens=[token.strip() for token in tokens]#去除单词前后的空格或换行字符
    return tokens
# 移除特殊字符
def remove_special_characters(text):
    tokens=tokenize_text(text)
    pattern=re.compile('[{}]'.format(re.escape(string.punctuation)))
    #re.compile函数根据包含的正则表达式的字符串创建模式对象,可以实现更有效率的匹配.
    #re.escape(pattern) 可以对字符串中所有可能被解释为正则运算符的字符进行转义的应用函数
    #string.punctuation找出字符串中的所有的标点
    filtered_tokens=filter(None,[pattern.sub(' ',token) for token in tokens])
    # filter过滤掉不符合条件的元素.pattern.sub实现比普通字符串replace 更加强大的替换功能
    filtered_text=' '.join(filtered_tokens)
    return filtered_text
# 移除停用词
def remove_stopwords(text):
    tokens=tokenize_text(text)
    filtered_tokens=[token for token in tokens
                    if token not in stopword_list]
    filtered_text=' '.join(filtered_tokens)
    return filtered_text
# keep text characters
def keep_text_characters(text):
    filtered_tokens=[]
    tokens=tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]',token):
            filtered_tokens.append(token)
    filtered_text=' '.join(filtered_tokens)
    return filtered_text
text = textString
text=remove_special_characters(text)
text=remove_stopwords(text)
text=keep_text_characters(text)
text=tokenize_text(text)
# normalized_corpus=[]
# normalized_corpus.append(text)
# normalized_corpus
# text=' '.join(text)
text

['Pernod',
 'takeover',
 'talk',
 'lifts',
 'Domecq',
 'Shares',
 'UK',
 'drinks',
 'food',
 'firm',
 'Allied',
 'Domecq',
 'risen',
 'speculation',
 'could',
 'target',
 'takeover',
 'France',
 'Pernod',
 'Ricard',
 'Reports',
 'Wall',
 'Street',
 'Journal',
 'Financial',
 'Times',
 'suggested',
 'French',
 'spirits',
 'firm',
 'considering',
 'bid',
 'yet',
 'contact',
 'target',
 'Allied',
 'Domecq',
 'shares',
 'London',
 'rose',
 'GMT',
 'Pernod',
 'shares',
 'Paris',
 'slipped',
 'Pernod',
 'said',
 'seeking',
 'acquisitions',
 'refused',
 'comment',
 'specifics',
 'Pernod',
 'last',
 'major',
 'purchase',
 'third',
 'US',
 'giant',
 'Seagram',
 'move',
 'propelled',
 'global',
 'top',
 'drinks',
 'firms',
 'The',
 'thirds',
 'Seagram',
 'bought',
 'market',
 'leader',
 'Diageo',
 'In',
 'terms',
 'market',
 'value',
 'Pernod',
 '5bn',
 'euros',
 '7bn',
 'smaller',
 'Allied',
 'Domecq',
 'capitalisation',
 '7bn',
 '7bn',
 '2bn',
 'euros',
 'Last',
 'year',
 'Pernod',
 'tried',
 '

## k-shingles

In [3]:
from nltk.util import ngrams
shingling_2=list(ngrams(text, 20))
shingling_3=list(ngrams(text, 30))
shingling_4=list(ngrams(text, 40))
# shingling_5=list(ngrams(text, 11))
data=shingling_2+shingling_3+shingling_4

## 利用mlxtend包进行挖掘频繁项

In [4]:
# from apyori import apriori
# result= list(apriori(transactions=data,min_support=0.4))
# print(result)
# print(data)
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# 测试数据
# data = [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
# data = [('豆奶','莴苣'),
#         ('莴苣','尿布','葡萄酒','甜菜'),
#         ('豆奶','尿布','葡萄酒','橙汁'),
#         ('莴苣','豆奶','尿布','葡萄酒'),
#         ('莴苣','豆奶','尿布','橙汁')]
# def encode_units(x):
#     if x <= 0:
#         return 0
#     if x >= 1:
#         return 1
# data = data.applymap(encode_units)
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
a_rules = association_rules(frequent_itemsets, min_threshold=0.1)
frequent_itemsets

,support,itemsets
0,0.727941,(Allied)
1,0.691176,(Domecq)
2,0.862745,(Pernod)
3,0.669118,"(Domecq, Allied)"
4,0.637255,"(Pernod, Allied)"
5,0.595588,"(Domecq, Pernod)"
6,0.578431,"(Domecq, Pernod, Allied)"


In [5]:
a_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Domecq),(Allied),0.691176,0.727941,0.669118,0.968085,1.329895,0.165982,8.524510
1,(Allied),(Domecq),0.727941,0.691176,0.669118,0.919192,1.329895,0.165982,3.821691
2,(Pernod),(Allied),0.862745,0.727941,0.637255,0.738636,1.014692,0.009227,1.040921
3,(Allied),(Pernod),0.727941,0.862745,0.637255,0.875421,1.014692,0.009227,1.101749
4,(Domecq),(Pernod),0.691176,0.862745,0.595588,0.861702,0.998791,-0.000721,0.992459
5,(Pernod),(Domecq),0.862745,0.691176,0.595588,0.690341,0.998791,-0.000721,0.997302
6,"(Domecq, Pernod)",(Allied),0.595588,0.727941,0.578431,0.971193,1.334165,0.144878,9.444328
7,"(Domecq, Allied)",(Pernod),0.669118,0.862745,0.578431,0.864469,1.001998,0.001153,1.012719
8,"(Pernod, Allied)",(Domecq),0.637255,0.691176,0.578431,0.907692,1.313257,0.137976,3.345588
9,(Domecq),"(Pernod, Allied)",0.691176,0.637255,0.578431,0.836879,1.313257,0.137976,2.223785


## Ariori实现频繁项挖掘

In [6]:
def generate_freq_supports(data_set,item_set,min_support):
    freq_set = set() # 保存频繁项集元素
    item_count = {} # 保存元素频次，用来计算支持度
    supports = {} # 保存支持度
    # 如果项集中的元素再数据集中则计数
    for record in data_set:
        for item in item_set:
            if item.issubset(record):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1
    data_len = float(len(data_set))
    # 计算项集的支持度
    for item in item_count:
        if (item_count[item]/data_len) >= min_support:
            freq_set.add(item)
            supports[item] = item_count[item]/data_len
            
    return freq_set,supports

# 生成新组合，由初始候选集生成
def generate_new_combinations(freq_set,k):
    new_combinations = set() # 保存新组合
    sets_len = len(freq_set) # 集合含有元素的个数，用于遍历求得组合
    freq_set_list = list(freq_set) #集合转为列表用于索引
    
    for i in range(sets_len):
        for j in range(i+1,sets_len):
            l1 = list(freq_set_list[i])
            l2 = list(freq_set_list[j])
            l1.sort()
            l2.sort()
            # 项集若有相同的父集则合并项集
            if l1[0:k-2] == l2[0:k-2]:
                freq_item = freq_set_list[i]|freq_set_list[j]
                new_combinations.add(freq_item)
    return new_combinations
def apriori(data_set,min_support,max_len=None):
    max_items = 2 # 初始项集元素的个数
    freq_sets = [] # 保存所有的频繁项集
    supports = {} # 保存所有的支持度
    # 候选项1项集
    c1 = set()
    for items in data_set:
        for item in items:
            item_set = frozenset([item])#frozenset() 冻结后集合不能再添加或删除任何元素。为了后续计算支持度字典是将集合作为键
            c1.add(item_set) 
#     print('c1:',c1)
#     print('\n')
    # 频繁项1项集及其支持度
    l1,support1 = generate_freq_supports(data_set,c1,min_support)
    freq_sets.append(l1)
    supports.update(support1)
#     print('频繁项1项集:',freq_sets[-1])
#     print('频繁项1项集:',freq_sets)
#     print('支持度:',supports)
#     print('\n')
    
    if max_len is None:
        max_len = float('inf')
        
    while max_items and max_items <= max_len:
        ci = generate_new_combinations(freq_sets[-1],max_items) #生成候选集
        li,support = generate_freq_supports(data_set,ci,min_support) # 生成频繁项集和支持度
#         print('生成候选集 ci',ci)
#         print('\n')
        # 如果由频繁项及则进入下个循环
        if li:
            freq_sets.append(li)
            supports.update(support)
            max_items += 1
        else:
            max_items = 0
    return freq_sets,supports    
def association_reles(freq_sets,supports,min_conf):
    rules = []
    max_len = len(freq_sets)
    
    # 生成关联规则，筛选符合规则的频繁项集计算置信度，满足最小置信度的关联规则添加到列表
    for k in range(max_len - 1):
        for freq_set in freq_sets[k]:
            for sub_set in freq_sets[k + 1]:
                if freq_set.issubset(sub_set):
                    conf = supports[sub_set] / supports[freq_set]
                    rule = (freq_set, sub_set - freq_set, conf)
                    if conf >= min_conf:
                        rules.append(rule)
    return rules
L,support_data = apriori(data, min_support=0.5)
rules = association_reles(L,support_data,0.1)
# print(L)
support_data

{frozenset({'Pernod'}): 0.8627450980392157,
 frozenset({'Domecq'}): 0.6911764705882353,
 frozenset({'Allied'}): 0.7279411764705882,
 frozenset({'Allied', 'Domecq'}): 0.6691176470588235,
 frozenset({'Domecq', 'Pernod'}): 0.5955882352941176,
 frozenset({'Allied', 'Pernod'}): 0.6372549019607843,
 frozenset({'Allied', 'Domecq', 'Pernod'}): 0.5784313725490197}

In [7]:
rules

[(frozenset({'Domecq'}), frozenset({'Allied'}), 0.9680851063829786),
 (frozenset({'Domecq'}), frozenset({'Pernod'}), 0.8617021276595744),
 (frozenset({'Pernod'}), frozenset({'Domecq'}), 0.6903409090909091),
 (frozenset({'Pernod'}), frozenset({'Allied'}), 0.7386363636363635),
 (frozenset({'Allied'}), frozenset({'Domecq'}), 0.9191919191919191),
 (frozenset({'Allied'}), frozenset({'Pernod'}), 0.8754208754208754),
 (frozenset({'Allied', 'Domecq'}), frozenset({'Pernod'}), 0.8644688644688646),
 (frozenset({'Domecq', 'Pernod'}), frozenset({'Allied'}), 0.9711934156378602),
 (frozenset({'Allied', 'Pernod'}), frozenset({'Domecq'}), 0.9076923076923078)]